In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, Conv1D, MaxPooling1D
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
#import pandas as pd
#import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec
import talos as ta
from keras.layers import Dropout
from talos.model.early_stopper import early_stopper
from talos.model.normalizers import lr_normalizer
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
#import seaborn as sns
#import matplotlib.pyplot as plt

%matplotlib inline

ImportError: cannot import name 'ft2font' from partially initialized module 'matplotlib' (most likely due to a circular import) (/usr/lib/python3/dist-packages/matplotlib/__init__.py)

In [ ]:
df = pd.read_csv('../csv/updated_target_merged_data.csv')
df.head()

In [ ]:
# Scale Tweet Length
scaler = StandardScaler()
df_2 = df.drop(columns = ['Retweets'])
df_2[['Tweet Length']] = scaler.fit_transform(df[['Tweet Length']])
df_2.head()

In [ ]:
target = df.Signal
y = pd.get_dummies(target).values

print(type(y))
y[:4]

In [ ]:
df_no_tweet = df_2.drop(['Date', 'Tweet', 'Volume', 'Gap', 'Signal'], axis=1)
df_no_tweet.head()

In [ ]:
# Features for the no_tweet ANN model
X = df_no_tweet.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

classifier = Sequential()
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'tanh'))
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'tanh'))
classifier.add(Dense(units = 3, kernel_initializer='uniform', activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
c = classifier.fit(X_train, y_train, batch_size = 10, epochs = 4, validation_split=0.1)

In [ ]:
score_no_tweet, acc_no_tweet = classifier.evaluate(X_test, y_test, verbose = 3, batch_size = 32)
print("score for the test data no tweet model: %.3f" % (score_no_tweet))
print("acc for the test data  no tweet model: %.7f" % (acc_no_tweet))

score_train_no_tweet, acc_train_no_tweet = classifier.evaluate(X_train, y_train, verbose = 3, batch_size = 32)
print("score for the training data no tweet model: %.3f" % (score_train_no_tweet))
print("acc for the training data no tweet model: %.7f" % (acc_train_no_tweet))

In [ ]:
def plot_train_val(model, label):
    plt.plot(model.history[label]) 
    plt.plot(model.history['val_' + label])
    plt.title('model '+ label)
    plt.ylabel(label)
    plt.xlabel('epoch')
    plt.gca().legend(('train','val'))
    return plt.show()

In [ ]:
plot_train_val(c, 'acc'), plot_train_val(c, 'loss')